In [2]:
import tweepy
import csv
import pandas as pd
from tweepy.streaming import StreamListener
from tweepy import Stream
import time

In [3]:
#input your credentials here
consumer_key = "XXXXXXXX"
consumer_secret = "XXXXXXXX"
access_token = "XXXXXXXX"
access_token_secret = "XXXXXXXX"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
csvFile = open('ua7.csv', 'a')
csvWriter = csv.writer(csvFile)
                
df= pd.DataFrame()
vdf= pd.DataFrame()

# Creation of lists to add to dataframe
dates = []
tweetid = []
tweettext = []
tweetcoordinates = []
usercoordinates = []
username = []

     
#iterating through hashtags to fetch tweets 

# Parameters for function: enter latitude, longitude of area you want
def find_relevant_tweets(lat,long):
    count=0
    try:
        # Will search specific words or hashtags inside radius around chosen lat/long
        word_list = ['#wildfires']
        for word in word_list:
            for tweet in tweepy.Cursor(api.search,
                                        # Latitude and longitude variables are used to designate center point middle of radius
                                        # Can modify mi radius here, currently set to 50 mile radius around center point
                                       q=word+'\ngeocode:"'+str(lat)+','+str(long)+',50mi"',
                                       # Currently only scrapes for 3k tweets
                                       count=3000,
                                       # Can change language of items
                                       lang="en",
                                       since="2019-10-27").items():
                
                # Only scrapes for tweets and not retweets, can remove this if you want to capture all
                if (not tweet.retweeted) and ('RT @' not in tweet.text):

                    count+=1
                    if count >3000:
                        break
                    dates.append(tweet.created_at)
                    tweetid.append(tweet.id)
                    tweettext.append(tweet.text)
                    # Check statement for if there are tweet coordinates associated with tweets
                    # If there are coordinates it tracks them and adds to list
                    if (tweet.coordinates!=None):
                        tweetcoordinates.append(tweet.coordinates)
                    # Else if there are no coordinates with the tweets 
                    else:
                        tweetcoordinates.append(tweet.coordinates)
                    username.append(tweet.user.screen_name)
                    print(tweet.user.screen_name)
#                     urls.append(tweet.entities['urls'])
#                     print(tweepy.get_user(tweet.user))
                    
    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(5)

    # Creation of dataframe with lists
    vdf['Date']= dates
    vdf['tweet id']= tweetid
    vdf['tweet text']=tweettext
    vdf['Geolocation']=tweetcoordinates
    vdf['Username']=username
    
    # Checks for dubplicate tweets
    result_df1 = vdf.drop_duplicates(subset=['tweet text'], keep='first')

    return result_df1

#writing dataframe to csv
# vdf.to_csv('firetweets.csv', encoding='utf-8')

In [ ]:
# creates new dataframe from function
vdf = find_relevant_tweets(34.328,-119.008)

In [11]:
# Geolocation data and urls are stored in json may need cleaning
print("Total tweets:", vdf.shape[0])
print("Tweets with geolocations:", vdf[~vdf['Geolocation'].isna()].shape[0])

# Check if any entries have na values in geolocation
vdf[~vdf['Geolocation'].isna()]

Total tweets: 1403
Tweets with geolocations: 20


,Date,tweet id,tweet text,Geolocation,urls
105,2019-11-02 04:55:17,1190492551901351937,Some scenes of Friday’s firefight in Saticoy a...,"{'type': 'Point', 'coordinates': [-119.068, 34...","[{'url': 'https://t.co/807Txwbt6X', 'expanded_..."
136,2019-11-02 02:24:50,1190454690779025408,Smoke from Maria fire has drifted into our val...,"{'type': 'Point', 'coordinates': [-120.088, 34...","[{'url': 'https://t.co/K2N5t39Q7v', 'expanded_..."
166,2019-11-02 00:29:21,1190425627297538049,"Esteban Minero, Katelyn Magee, #Farmworkers, #...","{'type': 'Point', 'coordinates': [-119.068, 34...","[{'url': 'https://t.co/Zc5I5Z1UsF', 'expanded_..."
171,2019-11-02 00:12:01,1190421268215943168,The #MariaFire #Camarillo #California #PhotogB...,"{'type': 'Point', 'coordinates': [-119.0702346...","[{'url': 'https://t.co/KJyEjUXTLZ', 'expanded_..."
187,2019-11-01 23:02:54,1190403872247955456,Hopefully this is the last of it....Thinking o...,"{'type': 'Point', 'coordinates': [-119.068, 34...","[{'url': 'https://t.co/pBxebaIiff', 'expanded_..."
203,2019-11-01 22:14:45,1190391756384280576,MSB Disaster Recovery Services is here when yo...,"{'type': 'Point', 'coordinates': [-119.068, 34...","[{'url': 'https://t.co/ceEZktE8yc', 'expanded_..."
263,2019-11-01 19:55:07,1190356615687335936,"#11-1 1 pm. \nRetardant is slowing, but not s...","{'type': 'Point', 'coordinates': [-119.068, 34...","[{'url': 'https://t.co/RiAkBQ1qM8', 'expanded_..."
267,2019-11-01 19:48:12,1190354874724241409,"11-1 12:45 pm. \nTodd Road, Santa Paul, Ca \n...","{'type': 'Point', 'coordinates': [-119.068, 34...","[{'url': 'https://t.co/dderGzwseZ', 'expanded_..."
327,2019-11-01 18:54:32,1190341369593597952,#Mariafire #Saticoy #Somis #SantaPaula @ Camar...,"{'type': 'Point', 'coordinates': [-119.0391, 3...","[{'url': 'https://t.co/PZjnteSA4G', 'expanded_..."
348,2019-11-01 18:38:13,1190337263026102272,Current view of #MariaFire from hospital hill ...,"{'type': 'Point', 'coordinates': [-119.0621728...",[]


In [12]:
# Creates new dataframe that don't have any na values in geolocation
geolocated_data = vdf[~vdf['Geolocation'].isna()]

In [80]:
# Function to pull the url alone instead of json values
def pull_url(row):
    if(len(row.urls)>0):
        return row.urls[0]['expanded_url']
    

In [81]:
# Applies function to all urls and creates new column called pulled
geolocated_data['pulled'] = geolocated_data.apply(pull_url, axis=1)

C:\Users\marti\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# Removes all entries that have na values in the new pulled url column
geolocated_data = geolocated_data[~geolocated_data['pulled'].isna()]

In [98]:
# Drops url column and keeps pulled column that contains cleaned urls
geolocated_data.drop('urls',axis=1)

,Date,tweet id,tweet text,Geolocation,pulled
105,2019-11-02 04:55:17,1190492551901351937,Some scenes of Friday’s firefight in Saticoy a...,"{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190492551901...
136,2019-11-02 02:24:50,1190454690779025408,Smoke from Maria fire has drifted into our val...,"{'type': 'Point', 'coordinates': [-120.088, 34...",https://twitter.com/i/web/status/1190454690779...
166,2019-11-02 00:29:21,1190425627297538049,"Esteban Minero, Katelyn Magee, #Farmworkers, #...","{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190425627297...
171,2019-11-02 00:12:01,1190421268215943168,The #MariaFire #Camarillo #California #PhotogB...,"{'type': 'Point', 'coordinates': [-119.0702346...",https://www.instagram.com/p/B4V6x1enFuf/?igshi...
187,2019-11-01 23:02:54,1190403872247955456,Hopefully this is the last of it....Thinking o...,"{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190403872247...
203,2019-11-01 22:14:45,1190391756384280576,MSB Disaster Recovery Services is here when yo...,"{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190391756384...
263,2019-11-01 19:55:07,1190356615687335936,"#11-1 1 pm. \nRetardant is slowing, but not s...","{'type': 'Point', 'coordinates': [-119.068, 34...",https://www.instagram.com/p/B4VdNP4AOD9/?igshi...
267,2019-11-01 19:48:12,1190354874724241409,"11-1 12:45 pm. \nTodd Road, Santa Paul, Ca \n...","{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190354874724...
327,2019-11-01 18:54:32,1190341369593597952,#Mariafire #Saticoy #Somis #SantaPaula @ Camar...,"{'type': 'Point', 'coordinates': [-119.0391, 3...",https://www.instagram.com/p/B4VWXDflRYN/?igshi...
402,2019-11-01 17:21:02,1190317838306828288,"The #MariaFire broke out in Santa Paula, CA la...","{'type': 'Point', 'coordinates': [-119.13599, ...",https://twitter.com/i/web/status/1190317838306...


In [113]:
final_output = geolocated_data[['Date','Geolocation','pulled']]

In [115]:
final_output.head()

,Date,Geolocation,pulled
105,2019-11-02 04:55:17,"{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190492551901...
136,2019-11-02 02:24:50,"{'type': 'Point', 'coordinates': [-120.088, 34...",https://twitter.com/i/web/status/1190454690779...
166,2019-11-02 00:29:21,"{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190425627297...
171,2019-11-02 00:12:01,"{'type': 'Point', 'coordinates': [-119.0702346...",https://www.instagram.com/p/B4V6x1enFuf/?igshi...
187,2019-11-01 23:02:54,"{'type': 'Point', 'coordinates': [-119.068, 34...",https://twitter.com/i/web/status/1190403872247...


In [114]:
final_output.to_csv('final_output.csv')